In [1]:
import requests
import pandas as pd
from datetime import date, datetime, timedelta

In [2]:
response = requests.get("https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQCB/securities.xml?iss.meta=off&iss.only=securities&securities.columns=SECID,SECNAME,SHORTNAME,COUPONVALUE,COUPONPERCENT,MATDATE,NEXTCOUPON,STATUS,FACEVALUE")

In [3]:
bonds = pd.read_xml(response.content, xpath=".//row")

In [4]:
bonds

,SECID,SECNAME,SHORTNAME,COUPONVALUE,COUPONPERCENT,MATDATE,NEXTCOUPON,STATUS,FACEVALUE
0,AMUNIBB2DER6,UNIBANK OJSC Series 1 21,UBANK02/24,1.33,5.300,2024-02-05,2023-02-05,A,100.0
1,RU000A0JQ7Z2,"""Российские ЖД"" ОАО 19 обл.",РЖД-19 обл,39.14,7.850,2024-07-08,2023-01-09,N,1000.0
2,RU000A0JQAL8,ДОМ.РФ (АО) обл. сер. А14,ДОМ.РФ14об,49.59,10.000,2023-05-15,2023-05-15,A,1000.0
3,RU000A0JQAM6,ДОМ.РФ (АО) обл. сер. А15,ДОМ.РФ15об,52.07,10.500,2028-09-15,2023-03-15,A,1000.0
4,RU000A0JQRD9,"""Российские ЖД"" ОАО 23 обл.",РЖД-23 обл,39.14,7.850,2025-01-16,2023-01-19,A,1000.0
...,...,...,...,...,...,...,...,...,...
2037,XS2392969395,CBOM Finance P.L.C. 7.625 UNDT,CBOM Finan,19.06,7.625,0000-00-00,2023-01-04,A,1000.0
2038,XS2393688598,SovCom Capital UNTD,SCB-UNTD,19.00,7.600,0000-00-00,2023-02-17,A,1000.0
2039,XS2415466676,BCS SP Plc Series 229,BCS11/24-8,10.00,0.010,2024-11-29,2023-11-29,A,100000.0
2040,XS2417535650,BCS SP Plc Series 234,BCS06/25-A,0.00,NaN,2025-06-01,2023-12-01,A,100000.0


In [5]:
bonds.to_csv("1.csv")

In [6]:
bonds = bonds.dropna()
bonds = bonds.drop(
    bonds[
        (bonds.COUPONPERCENT < 6)
        | (bonds.COUPONVALUE == 0)
        | (bonds.MATDATE == "0000-00-00")
        | (bonds.NEXTCOUPON == "0000-00-00")
        | (bonds.STATUS != "A")
        | (bonds.FACEVALUE != 1000)
    ].index
)


In [7]:
bonds["MATDATE"]

2       2023-05-15
3       2028-09-15
4       2025-01-16
7       2031-05-13
8       2031-05-13
           ...    
1958    2027-06-29
1967    2030-02-28
1979    2024-06-25
1982    2030-04-07
1999    2031-02-24
Name: MATDATE, Length: 995, dtype: object

In [8]:
def delta(d):
    d = datetime.strptime(d["MATDATE"], '%Y-%m-%d').date()
    delta = d - date.today()
    y = delta.days // 365
    m = delta.days % 365 // 30
    if y > 0:
        return f"{y} years {m} months"
    return f"{m} months"

In [9]:
bonds["DELTA"] = bonds.apply(delta, axis=1)

In [12]:
bonds.sort_values("COUPONPERCENT").to_excel("/1.xlsx")